In [1]:
import random

# Problem: Traveling Salesman Problem
CUSTOMERS = [(0, 0), (1, 5), (2, 2), (3, 8), (5, 5), (6, 1), (8, 3)]

POPULATION_SIZE = 4  # Smaller population for demonstration
GENERATIONS = 2
VEHICLES = 3

def create_individual():
    """
    Creates a random individual for a multi-vehicle TSP/VRP.
    Representation: List of lists, one list per vehicle containing assigned customers.
    Allows uneven distribution of customers.
    """
    customers = list(range(1, len(CUSTOMERS)))  # exclude depot (index 0)
    random.shuffle(customers)

    individual = [[] for _ in range(VEHICLES)]

    for cust in customers:
        chosen_vehicle = random.randint(0, VEHICLES - 1)  # assign randomly
        individual[chosen_vehicle].append(cust)

    return individual

In [ ]:
# Testing creation of individuals
INDIVIDUALS = []
for i in range(POPULATION_SIZE):
  INDIVIDUALS.append(create_individual())

INDIVIDUALS

[[[3, 2], [1, 6], [4, 5]],
 [[4], [3, 1], [2, 5, 6]],
 [[3, 5, 4], [6, 1], [2]],
 [[4], [6, 5], [2, 1, 3]]]